In [1]:
# Importing libraries 
import pandas as pd
import os
from  statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller


In [2]:
# Setting the relative path


In [3]:
# Importing data


In [4]:
# Data cleaning & merging
oil_price = pd.read_csv('../data/DCOILBRENTEU.csv').rename(columns = {'DATE':'Date'}).assign(Date= lambda oil_price: pd.to_datetime(oil_price['Date']))

sp500 =  pd.read_csv('../data/SP500.csv').rename(columns = {'DATE':'Date'}).assign(Date= lambda oil_price: pd.to_datetime(oil_price['Date']))
def extract_stock_price(stock):
    '''This functions takes stock price from dataset and import a cleaned dataframe '''
    file_path = os.path.join("data", stock+'.csv')
    file_folder = pd.read_csv(file_path).assign(Date = lambda stock: pd.to_datetime(stock['Date']))
    return file_folder
tsla = extract_stock_price('TSLA')
nvda = extract_stock_price('NVDA')

def merge_dataset(data1, data2):
    # merge two dataset based on Date
    new_data_frame = pd.merge(data1, data2, on = ['Date'])
    return new_data_frame


oil_sp = merge_dataset(oil_price, sp500)
oil_sp



In [5]:
# Data Analysis

## In this part you can include your codes to analyse the data,we'll start with (MA, VAT, Trend analysis, ...etc).
## You may also add the visualisation codes to your analysis.